# load data

In [1]:
# import the necessary packages
import imutils
import matplotlib.pyplot as plt

import os
import numpy as np
from scipy import ndimage
from tqdm import tqdm  # ! this might result into problem with 'object'
import pandas as pd
import random
import glob
from skimage import io

import argparse
import time
import cv2

In [2]:
# read in the raw image

image = np.load('./data/rawGray15.npy')
print(image.shape)

(15, 20, 1040, 1392)


# The multi-scanning pipeline on test dataset

In [3]:
# create the folder to save the stacks

import os

def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)           
#     else:
#         print('path existed')

## scan for the multiple images

In [4]:
# multiple scans for one image.
# input: stack num, slice num, image, step, windowsize

def slidingStack(stackNum, sliceNum, image, stepArray, windowSize, savedPath):
    
    focalStack = []
    
    for i in range(len(stepArray)):
        
        stepSize = stepArray[i]
        # slide a window across the image
        stdMap = np.zeros((image.shape[:2]))  # careful the image is 3 channels

        for y in range(0, image.shape[0], stepSize):
            for x in range(0, image.shape[1], stepSize):
                # yield the current window
                yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
                tPatch = image[y:y + windowSize[1], x:x + windowSize[0], 0]  # taking only 1 channel from 3
                tempStdPatch = np.ones((tPatch.shape)) * np.std((tPatch))
                # print(tempStdPatch)
                stdMap[y:y + windowSize[1], x:x + windowSize[0]] = tempStdPatch + stdMap[y:y + windowSize[1], x:x + windowSize[0]] 
                # print(tempStd)
                
        focalStack.append(stdMap)
        
    # calculate the stack and conduct max projection
    fMax = np.max(np.asarray(focalStack), axis=0)
    
    # check the saving path
    tempPath = savedPath + '/' + str(stackNum)
    mkdir(tempPath)
    
    # save the max projection in sliceNum under the path 'stackNum'
    np.save(tempPath+'/'+str(sliceNum)+'.npy', fMax)

In [5]:
def visualizeStack(image, stepArray, windowSize):
    
    # slide window scanning for one test
    for (x, y, window) in slidingStack(image, stepArray=stepArray, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if window.shape[0] != winH or window.shape[1] != winW:
            continue
        # draw the window during the process
        clone = image.copy() 
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)  # third param indicates the colour
        cv2.imshow("Window", clone)
        cv2.waitKey(1)
           # time.sleep(0.025)
        time.sleep(0.00025)
    cv2.destroyAllWindows() 

In [6]:
# f-max is the max projection of the stacks

# image = np.load('./notebooks/fMaxTest.npy')
# print(image.shape)
# test_slice = image[0,19,...]
# plt.imshow(test_slice, cmap='gray')

In [7]:
# sorting out the image
# rawData = np.stack((image,)*3, axis=-1)  # remember the operation is on 3 channels
rawData = image
print(rawData.shape)
# plt.imshow(image, cmap='gray')

(15, 20, 1040, 1392)


In [8]:
# define the scanning parameters

# step size
# stepArray = [22, 24]
stepArray = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]
# window size
# (winW, winH) = (256, 256)
(winW, winH) = (128, 128)
savedPath = './data/dataPre'
# target stack
targetStack = rawData
# targetStack = tempStack[:,:2,:,:]

In [9]:
# scan the images in step size
from timeit import default_timer as timer
start = timer()

for stackNum in range(targetStack.shape[0]):
    
    for sliceNum in range(targetStack.shape[1]):
        image = np.stack((targetStack[stackNum,sliceNum,...],)*3, axis=-1) 
        for (x, y, window) in slidingStack(stackNum, sliceNum, image, stepArray=stepArray, windowSize=(winW, winH), savedPath=savedPath):
        # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue 
                
end = timer()

KeyboardInterrupt: 

In [ ]:
time1_local_scan = end-start
print('the original scanning:', time1_local_scan)

## sort out the images from the max projection

In [ ]:
# define the read in function for max projection

def readTIF(data_path):
    fStack = []
    for img in glob.glob(os.path.join(data_path, "*.npy")):
        im = np.load(os.path.join(data_path, img))  
        imarray = np.array(im)
        fStack.append(imarray)  # images, num of slices, channels, img.x, img.y
    return np.asarray(fStack)

In [ ]:
# read in multiple stacks

fStacks = []

for i in range(tempStack.shape[0]):
    fStackTemp = readTIF(savedPath+'/'+str(i)+'/')
    fStacks.append(fStackTemp)
    
fStacks = np.asarray(fStacks)

# The time consuming for all features

In [3]:
# create the folder to save the stacks
import os
from timeit import default_timer as timer

def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)           
    # else:
    #     print('path existed')

In [4]:
# various blur detectors
# input patch-uint8

import numpy as np
from skimage import filters
import os
import cv2

# brenner detector
def brenner(img):
    
    imgMat = img/255.0
    
    x, y = imgMat.shape
    score = 0
    for i in range(x - 2):
        for j in range(y - 2):
            score += (imgMat[i + 2, j] - imgMat[i, j]) ** 2

    score = score/10

    return score

# tenengrad detector
def Tenengrad(img):

    f = img
    tmp = filters.sobel(f)
    source = np.sum(tmp**2)
    source = np.sqrt(source)

    return source

# laplacian
# laplacian
def  laplacian(img):

    resLap = cv2.Laplacian(np.uint8(img), cv2.CV_8U)  # adapt to the images, ddid not change the distributions
    score = resLap.var()
    return score

# SMD
def SMD(img):

    f=img / 255.0 # [0,1]
    x, y = f.shape
    score = 0
    for i in range(x - 1):
        for j in range(y - 1):
            score += np.abs(f[i+1,j]-f[i,j])+np.abs(f[i,j]-f[i+1,j])
    score=score/100
    return score

# SMD2
def SMD2(img):

    f = img/255.0
    x, y = f.shape
    score = 0
    for i in range(x - 1):
        for j in range(y - 1):
            score += np.abs(f[i+1,j]-f[i,j])*np.abs(f[i,j]-f[i,j+1])

    return score

# Variance
def Variance(img):

    f = img
    score = np.var(f)/1000
    
    return score

# Vollath
def Vollath(img):

    f = img
    source = 0
    x,y = f.shape
    for i in range(x-1):
        for j in range(y):
            source += f[i,j]*f[i+1,j]
    source = source - x*y*np.mean(f)
    source = source/100000
    return source

# rescale the image for comparing
def rescaleIMG(img, MIN, MAX):
    temp = np.interp(img, (img.min(), img.max()), (MIN, MAX))
    return temp

In [5]:
# multiple scans for one image with quality detectors.
# input: stack num, slice num, image, step, windowsize, metric: indicate the metric for detecting

import numpy as np
from skimage import filters
import os
import cv2

# one image
def slidingStackQuality(stackNum, sliceNum, image, stepArray, windowSize, savedPath, metric):
    
    focalStack = []
    
    for i in range(len(stepArray)):
        
        stepSize = stepArray[i]
        # slide a window across the image
        feaMap = np.zeros((image.shape[:2]))  # careful the image is 3 channels

        for y in range(0, image.shape[0], stepSize):
            for x in range(0, image.shape[1], stepSize):
                # yield the current window
                yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
                
                tPatch = image[y:y + windowSize[1], x:x + windowSize[0], 0]  # taking only 1 channel from 3
                
                #########
                if metric == 'std':
                    # Roy. calculate the std
                    tempPatch = np.ones((tPatch.shape)) * np.std((tPatch))
                    
                elif metric == 'brenner':
                    # print('cal robert', tPatch.shape, tPatch.min(), tPatch.max())
                    tempPatch = np.ones((tPatch.shape)) * np.round(brenner(tPatch), 2)
                    # print('robert mean:',tempPatch.mean())
                    
                elif metric == 'laplacian':
                    tempPatch = np.ones((tPatch.shape)) * np.round(laplacian(tPatch), 2)
                    
                elif metric == 'tenengrad':
                    tempPatch = np.ones((tPatch.shape)) * np.round(Tenengrad(tPatch), 2)
                    # print('raw tenengrad:', np.unique(tempPatch))
                    
                elif metric == 'SMD':
                    tempPatch = np.ones((tPatch.shape)) * np.round(SMD(tPatch), 2)
                    
                elif metric == 'SMD2':
                    tempPatch = np.ones((tPatch.shape)) * np.round(SMD2(tPatch), 2)
                    
                elif metric == 'Variance':
                    tempPatch = np.ones((tPatch.shape)) * Variance(tPatch)
                    
                    
                elif metric == 'Vollath':
                    tempPatch = np.ones((tPatch.shape)) * np.round(Vollath(tPatch), 2)
                
                #########
                feaMap[y:y + windowSize[1], x:x + windowSize[0]] = tempPatch + feaMap[y:y + windowSize[1], x:x + windowSize[0]]
                
#         focalStack.append(stdMap)       
        # focalStack.append(feaMap[padH:, padW:])  # crop the feature map back 
        focalStack.append(feaMap)
    
                           
    fMax = np.max(np.asarray(focalStack), axis=0)
    # plt.imshow(fMax, cmap='gray')
                           
    # check the saving path
    tempPath = savedPath + '/' + str(stackNum)
    mkdir(tempPath)
    
    # save the max projection in sliceNum under the path 'stackNum'
    np.save(tempPath+'/'+str(sliceNum)+'.npy', fMax)
    print('save sucessfully', sliceNum, tempPath)

In [6]:
intermediate_path = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/feature/intermediate/'

# train_norm_pad = np.load(intermediate_path + 'train_norm_pad.npy')
# val_norm_pad = np.load(intermediate_path + 'val_norm_pad.npy')
test_norm_pad = np.load(intermediate_path + 'test_norm_pad.npy')
print('test data size:', test_norm_pad.shape)

test data size: (2, 20, 1168, 1392)


In [7]:
# parameters for feature map

# step size
# stepArray = [8, 10]
stepArray = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]

# window size
(winW, winH) = (64*2, 64*2)

# pad parameters
(padW, padH) = (0, winH)

# saved path
# metricList = ['std', 'brenner', 'tenengrad', 'laplacian', 'SMD', 'SMD2', 'Variance', 'Vollath']  # brenner, SMD, SMD2, Vollath is slow
metric = 'SMD'

# savedPath = 'E:/LM/LM/digitalConfocal/notebooks/fMax/testResult/qualityDetect/' + str(metric)
# savedPath = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/feature/train/' + str(metric)
# savedPath = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/feature/test/' + str(metric)
# savedPath = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/feature/val/' + str(metric)
savedPath = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/test/' + str(metric)  # 测试扫描的参数
mkdir(savedPath)

# target data
# targetStack = val_norm_pad
# targetStack = train_norm_pad
targetStack = test_norm_pad
targetStack = targetStack[:1,...] # 拿出一个stack
# targetStack = image[:1,...]
print(targetStack.shape, targetStack.max())

(1, 20, 1168, 1392) 255


In [ ]:
# calculate the stack in metrics

start = timer()

for stackNum in range(targetStack.shape[0]):
    
    for sliceNum in range(targetStack.shape[1]):
        image = np.stack((targetStack[stackNum,sliceNum,...],)*3, axis=-1) 
        for (x, y, window) in slidingStackQuality(stackNum, sliceNum, image, stepArray=stepArray, windowSize=(winW, winH), savedPath=savedPath, metric = metric):
        # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue 
                
end = timer()

save sucessfully 0 /bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/test/SMD/0
save sucessfully 1 /bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_15_2_2/test/SMD/0


In [ ]:
print(str(metric), end-start)

# the DNN model for mask segmentation

In [ ]:
import os 
import numpy as np
from dataGenerator import imageLoader
import keras
import matplotlib.pyplot as plt
import glob
import random

In [ ]:
# visualize func

def visusalizeIMG(n_slice, test_img, test_msk):
    # n_slice = random.randint(0, test_img.shape[2])
    plt.figure(figsize=(8, 8))

    plt.subplot(121)
    plt.imshow(test_img[n_slice,:,:], cmap='gray')
    plt.title('image')
    plt.subplot(122)
    plt.imshow(test_msk[n_slice,:,:], cmap='gray')
    plt.title('mask')
    plt.show()
    
# visualization for two images

def subShow(IMG1, IMG2):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(IMG1, cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(IMG2, cmap='gray')
    plt.show()

In [ ]:
# reading into the corrected images for segmentation

PATH = '/bigdata/casus/MLID/RuiLi/Data/LM/zebrafish_partial_15/'

rawDataDNN = np.load(PATH + 'rawGray15_256.npy')
print(rawDataDNN.shape)  # print the size of the images

In [ ]:
# load in the trained model
import tensorflow as tf
from keras.models import load_model

# trained model name
MODEL_PATH = './models_weight/'
SVAED_MODEL_NAME = MODEL_PATH + 'simple2D_512' + '_' + '1000' + '.hdf5'

my_model = load_model(SVAED_MODEL_NAME, compile=False)


In [ ]:
test = rawDataDNN[:15]  # take the first 15 slices


In [ ]:
# prediction
pred_test = model.predict(img_test)
print(pred_test.shape, pred_test.min(), pred_test.max())